In [1]:
import os
import sys
from pathlib import Path
django_project_dir = Path(".")
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "newsilkroad.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

[raven.contrib.django.client.DjangoClient]DEBUG 17:51:58 Configuring Raven for host: None
[raven.contrib.django.client.DjangoClient]INFO 17:51:58 Raven is not configured (logging is disabled). Please see the documentation for more information.


In [2]:
import pandas as pd
import geopandas
# import matplotlib.pyplot as plt
from infrastructure.models import Project
from locations.models import GeometryStore
import ipdb

In [3]:
GeometryStore.objects.all().count()

10574

In [4]:
# sums = {
#     "project_id": None,
#     "overall_extent": None,
#     "collection_centroid": None,
#     "centroid": None,
#     "lines": _get_lines,
#     "polygons": None,
#     "points": None,
# }
# sums_list = []
# for p in Project.objects.filter(geo__isnull=False)[:2]:
#     row = sums.copy()
#     for attr in sums.keys():
#         if row[attr]:
#             row[attr] = row[attr](p)
#     print(row)
#     print()

In [5]:
def _get_lines(obj):
    if obj.geo.lines.exists():
        if obj.geo.lines.all().count() > 1:
            return ";".join([f"{line.geom}" for line in obj.geo.lines.all()])
        return f"{obj.geo.lines.all().first().geom}"

def _get_extents(obj):
    return f"{obj.geo.calculate_overall_extent()}"

def _get_centroid(obj):
    return f"{obj.geo.centroid}"

def _get_collection_centroid(obj):
    return f"{obj.get._get_collection_centroid()}"

def _get_polygons(obj):
    if obj.geo.polygons.exists():
        if obj.geo.polygons.all().count() > 1:
            return ";".join([f"{poly.geom}" for poly in obj.geo.polygons.all()])
        return f"{obj.geo.polygons.all().first().geom}"

def _get_points(obj):
    if obj.geo.points.exists():
        if obj.geo.points.all() > 1:
            return ";".join([f"{point.geom}" for point in obj.geo.points.all()])
        return f"{obj.geo.points.all().first().geom}"

In [34]:
from django.db import connection

df = geopandas.read_postgis("""select store.label AS label, store.identifier AS identifier, lines.geom AS geom 
FROM locations_geometrystore AS store, locations_linestringgeometry AS lines 
WHERE store.id = lines.id""", connection)
print(df)
df.to_file("shapefiles/lines/locations_lines.shp")
# def my_custom_sql():
#     columns = []
#     geometry_columns = {
#         'identifier': None,
#         'overall_extent': _get_extents, 
#         'centroid': _get_centroid, 
#         'collection_centroid': _get_collection_centroid, 
#         'lines': _get_lines, 
#         'polygons': _get_polygons, 
#         'points': _get_points
#     }
#     df = None
#     with connection.cursor() as cursor:
#         columns = {col[0] : None for col in cursor.description}
#         df = geopandas.read_postgis("select * from locations_linestringgeometry", )
#     print(df)

                                                  label  \
0     Lot 1 Svilaj on The Sava River (connection to ...   
1           Lot 2 Doboj South - Sarajevo South (Tarčin)   
2         Lanzhou-Chongqing Railway Development Project   
3                     Central Sichuan Roads Development   
4                       Zhengzhou-Xi’an Railway Project   
...                                                 ...   
3174                        Riccall Biomass Power Plant   
3175                                    WDF Power Plant   
3176                Granville Eco Park Biopower Project   
3177                Staples Vegetables Biopower Project   
3178                  Westcott Venture Park Power Plant   

                                identifier  \
0     c94f885b-4ba6-46d4-a02d-cbf1b8a80411   
1     eec75fa5-b7b7-49dd-ac49-1e8d24f6c3dd   
2     3b1671df-f213-4650-b21b-e3b6bb7f188c   
3     2ee1458b-22df-4593-8680-c08c816774ff   
4     cbf39312-2ce0-4cfc-9a06-2c1195e6fc49   
...            

ValueError: Invalid field type <class 'uuid.UUID'>

In [25]:
my_custom_sql()

         0                                                  1  \
0        1  Svilaj on The Sava River (connection to Corrid...   
1        2              Doboj South - Sarajevo South (Tarčin)   
2        3                  Lanzhou-Chongqing Railway Project   
3        4  Central Sichuan Roads Development Project In C...   
4        5  Central Sichuan Roads Development Project In C...   
...    ...                                                ...   
3297  3429                    Prijedor - Novi Grad - Dobrljin   
3298  3430                                   Vukosavlje-Brcko   
3299  3431             Oštarije – Škrljevo Section of Railway   
3300  3432  Modernization of Rijeka-Zagreb Rail (section O...   
3301  3433                                Karlovac - Oštarije   

                                                      2  \
0     {'name': 'Svilaj on The Sava River (connection...   
1     {'name': 'Doboj South - Sarajevo South (Tarčin...   
2     {'name': 'Lanzhou-Chongqing Railway 

AttributeError: No geometry data set yet (expected in column 'geometry'.)